In [ ]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

In [ ]:
bio_libraries <- c('CuratedAtlasQueryR','dittoSeq','Seurat','tidySingleCellExperiment')
libraries <- c('tidyverse','reshape2','scater','cowplot','scales','ggrepel', 'ggpubr','rstatix', 'scales', 'ggplot2')
lapply(bio_libraries,BiocManager::install, character.only = TRUE)
lapply(libraries, install.packages, character.only = TRUE)

In [ ]:
#Load libraries
lapply(libraries, library, character.only = TRUE)
lapply(bio_libraries, library, character.only = TRUE)

#Set working directory
cache_dir = "~/tmp" # specify the cache directory if you don't want to use default

#Load data
metadata <- get_metadata()
Overlapping_genes_alldb <- read.table("./results/common_IBD_genes.txt", quote="\"", comment.char="")

In [ ]:
#Filter data from sigmoid colon
features <- Overlapping_genes_alldb$V1

single_cell_counts <- metadata |>
                            dplyr::filter(
                                stringr::str_like(assay, "%10x%") &
                                tissue == "sigmoid colon"
                            ) |> get_seurat(features = features)

                            # get_SingleCellExperiment(assays = "cpm", features = as.character(features), cache_directory = cache_dir) 

single_cell_counts |> saveRDS("./results/single_cell_total_counts_sigmoid_colon_seurat.rds")
single_cell_counts <- readRDS("./results/single_cell_total_counts_sigmoid_colon_seurat.rds")

In [ ]:
# Visualize QC metrics as a violin plot
single_cell_counts[["percent.mt"]] <- PercentageFeatureSet(single_cell_counts, pattern = "^MT-")

VlnPlot(single_cell_counts, features = c("nFeature_originalexp", "nCount_originalexp", "percent.mt"), ncol = 3)
FeatureScatter(single_cell_counts, feature1 = "nCount_originalexp", feature2 = "nFeature_originalexp")

In [ ]:
#Analyze data from sigmoid colon
single_cell_counts <- NormalizeData(object = single_cell_counts)
single_cell_counts <- FindVariableFeatures(object = single_cell_counts)
single_cell_counts <- ScaleData(object = single_cell_counts)
single_cell_counts <- RunPCA(object = single_cell_counts)
single_cell_counts <- FindNeighbors(object = single_cell_counts, dims = 1:30)
single_cell_counts <- FindClusters(object = single_cell_counts)
single_cell_counts <- RunUMAP(object = single_cell_counts, dims = 1:30)

In [ ]:
#Save data
single_cell_counts |> saveRDS("./results/single_cell_total_counts_sigmoid_colon_seurat_analized.rds")
single_cell_counts <- readRDS("./results/single_cell_total_counts_sigmoid_colon_seurat_analized.rds")

In [ ]:
umap_sig_colon <- DimPlot(object = single_cell_counts, reduction = "umap", group.by = "cell_type_harmonised" )+
                          ggtitle("Sigmoidal Colon")+
                          theme_bw()
umap_sig_colon

pdf(file=paste0("./figures/","umap_sig_colon_total_cell_type.pdf"), width=7, height=6)
umap_sig_colon
dev.off()

In [ ]:
# Single cell heatmap of feature expression
single_cell_counts.markers <- FindAllMarkers(single_cell_counts, only.pos = TRUE)

single_cell_counts.markers %>%
                          group_by(cluster) %>%
                          slice_head(n = 10) %>%
                          ungroup() -> top10

heatmap1 <- DoHeatmap(subset(single_cell_counts, downsample = 100), features = features, group.by = "cell_type_harmonised")
heatmap1

pdf(file=paste0("./","heatmap_Sigmoidal_colon_IBD_genes_cell_type.pdf"), width=15, height=10)
heatmap1
dev.off()

In [ ]:
pdf(file=paste0("./figures/","UMAP_Sigmoidal_colon_top_10_IBD_genes.pdf"), width=15, height=10)
FeaturePlot(single_cell_counts, features = c("NOD2","ATGL16L1","IL23R", "IRGM", "TNFSF15", "IL10", "TLR4","VEGFA","PTPN2","STAT3"))
dev.off()